# Lightning With SPOT:

## 1. JSON


 `data.lightning_hyper_dict.json`

In [ ]:
  "GoogleNet":
    {
    "act_fn": {
            "levels": ["Sigmoid",
                       "Tanh",
                       "ReLU",
                       "LeakyReLU",
                       "ELU",
                       "Swish"],
            "type": "factor",
            "default": "ReLU",
            "transform": "None",
            "class_name": "spotPython.torch.activation",
            "core_model_parameter_type": "instance",
            "lower": 0,
            "upper": 5},
    "optimizer_name": {
        "levels": [
                   "Adam"
                ],
        "type": "factor",
        "default": "Adam",
        "transform": "None",
        "class_name": "torch.optim",
        "core_model_parameter_type": "str",
        "lower": 0,
        "upper": 0}
    }

## 2. Class HyperLightning.py

 ```{python}
 def fun(self,
        X: np.ndarray,
        fun_control: dict = None) -> np.ndarray:
```

1. Args: `numpy.array` from `spotPython`: hyperparameters as numerical values
2. Generates a dictionary with hyperparameters, e.g.:


   ```{JSON}
   config: {
      'act_fn': <class 'spotPython.torch.activation.ReLU'>,
      'optimizer_name': 'Adam'}`
   ```

3. Passes dictionary to method `train_model()` 

## 3. train_model()

 `def train_model(config: dict, fun_control: dict):`

1. Prepares the data, e.g., `CIFAR10DataModule`
2. Sets up the Trainer
3. Sets up the model, e.g.,

```{python}
model = NetCNNBase(
            model_name=fun_control["core_model"].__name__,
            model_hparams=config,
            optimizer_name="Adam",
            optimizer_hparams={"lr": 1e-3, "weight_decay": 1e-4},
        )
```

## 4. netcnnbase.py

```{python}
class NetCNNBase(L.LightningModule):
    def __init__(self,
                model_name,
                model_hparams,
                optimizer_name,
                optimizer_hparams):
```

1. Saves hyperparameters in `self.hparams`
2. Creates model
3. Creates loss module
4. Creates optimizer
5. Defines forward pass
6. Defines training step
7. Defines validation step
8. Defines test step

## 5. GoogleNet

```{python}
class GoogleNet(nn.Module):
    """GoogleNet architecture

    Args:
        num_classes (int):
            Number of classes for the classification task. Defaults to 10.
        act_fn_name (str):
            Name of the activation function. Defaults to "relu".
        **kwargs:
            Additional keyword arguments.

    Attributes:
        hparams (SimpleNamespace):
            Namespace containing the hyperparameters.
        input_net (nn.Sequential):
            Input network.
        inception_blocks (nn.Sequential):
            Inception blocks.
        output_net (nn.Sequential):
            Output network.

    Returns:
        (torch.Tensor):
            Output tensor of the GoogleNet architecture

    Examples:
        >>> from spotPython.light.cnn.googlenet import GoogleNet
            import torch
            import torch.nn as nn
            model = GoogleNet()
            x = torch.randn(1, 3, 32, 32)
            y = model(x)
            y.shape
            torch.Size([1, 10])
    """
```

## 6. InceptionBlock

```{python}
class InceptionBlock(nn.Module):
    def __init__(self, c_in, c_red: dict, c_out: dict, act_fn):
        """
        Inception block as used in GoogLeNet.

        Args:
            c_in:
                Number of input feature maps from the previous layers
            c_red:
                Dictionary with keys "3x3" and "5x5" specifying
                the output of the dimensionality reducing 1x1 convolutions
            c_out:
                Dictionary with keys "1x1", "3x3", "5x5", and "max"
            act_fn:
                Activation class constructor (e.g. nn.ReLU)

        Returns:
            torch.Tensor:
                Output tensor of the inception block

        Examples:
            >>> from spotPython.light.cnn.googlenet import InceptionBlock
                import torch
                import torch.nn as nn
                block = InceptionBlock(3,
                            {"3x3": 32, "5x5": 16},
                            {"1x1": 16, "3x3": 32, "5x5": 8, "max": 8},
                            nn.ReLU)
                x = torch.randn(1, 3, 32, 32)
                y = block(x)
                y.shape
                torch.Size([1, 64, 32, 32])

        """
```

## Tensorboard {#sec-tensorboard-31}

The textual output shown in the console (or code cell) can be visualized with Tensorboard.

```{raw}
tensorboard --logdir="runs/"

# Sample Run

In [3]:
from spotPython.utils.init import fun_control_init
from spotPython.utils.file import get_experiment_name, get_spot_tensorboard_path
from spotPython.utils.device import getDevice
from spotPython.light.cnn.googlenet import GoogleNet
from spotPython.data.lightning_hyper_dict import LightningHyperDict
from spotPython.hyperparameters.values import add_core_model_to_fun_control
from spotPython.fun.hyperlightning import HyperLightning
from spotPython.hyperparameters.values import get_default_hyperparameters_as_array

MAX_TIME = 1
INIT_SIZE = 3
WORKERS = 8
PREFIX="TEST"
experiment_name = get_experiment_name(prefix=PREFIX)
fun_control = fun_control_init(
    spot_tensorboard_path=get_spot_tensorboard_path(experiment_name),
    num_workers=WORKERS,
    device=getDevice(),
    _L_in=3,
    _L_out=10,
    TENSORBOARD_CLEAN=True)

add_core_model_to_fun_control(core_model=GoogleNet,
                            fun_control=fun_control,
                            hyper_dict= LightningHyperDict)

X_start = get_default_hyperparameters_as_array(fun_control)

hyper_light = HyperLightning(seed=126, log_level=50)
hyper_light.fun(X=X_start, fun_control=fun_control)

Global seed set to 42



core_model: <class 'spotPython.light.cnn.googlenet.GoogleNet'>
config: {'act_fn': <class 'spotPython.torch.activation.ReLU'>, 'optimizer_name': 'Adam'}
fun: Calling train_model
train_model: Starting
train_model: config: {'act_fn': <class 'spotPython.torch.activation.ReLU'>, 'optimizer_name': 'Adam'}
Files already downloaded and verified
Data mean [0.49139968 0.48215841 0.44653091]
Data std [0.24703223 0.24348513 0.26158784]
Files already downloaded and verified
Files already downloaded and verified


Global seed set to 42
Global seed set to 42


Files already downloaded and verified


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
Missing logger folder: runs/saved_models/GoogleNet/lightning_logs


...

model_hparams: {'act_fn': <class 'spotPython.torch.activation.ReLU'>, 'optimizer_name': 'Adam'}
self.hparams: "model_hparams":     {'act_fn': <class 'spotPython.torch.activation.ReLU'>, 'optimizer_name': 'Adam'}
"model_name":        GoogleNet
"optimizer_hparams": {'lr': 0.001, 'weight_decay': 0.0001}
"optimizer_name":    Adam
create_model: Starting
model_name: GoogleNet
model_hparams: {'act_fn': <class 'spotPython.torch.activation.ReLU'>, 'optimizer_name': 'Adam'}
self.model: GoogleNet(
  (input_net): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (inception_blocks): Sequential(
    (0): InceptionBlock(
      (conv_1x1): Sequential(
        (0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv_3x3): Sequent

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

fun: train_model returned


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
fun_control['core_model']
